In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
from pydub import AudioSegment
from pydub.playback import play

In [ ]:
class AudioFP():
    def __init__(self):
        self.songname = []
        
        ## Reading audio file using pydub
    def read_audiofile(self):
        self.songname = input("Enter filename: ")
    #     self.songname = "Choc--Eigenvalue-Subspace-Decomposition.mp3"
        audiofile = AudioSegment.from_file(self.songname)
        self.songdata = np.frombuffer(audiofile._data, np.int16)
        self.channels = []
        for chn in range(audiofile.channels):
            self.channels.append(self.songdata[chn::audiofile.channels])
        self.framerate = audiofile.frame_rate

In [ ]:
song1 = AudioFP()
song1.read_audiofile()